<a href="https://colab.research.google.com/github/greygrease/tf-stuff/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdunction to NLP in Tensorlfow

NLP has the goal of deriving information our of natural language (text or speech)

Commom NLP problems are sequence to sequence problems (seq2seq)

## Get helper function

In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-09-12 18:23:59--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-09-12 18:23:59 (110 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
# Import series of helper function for the notebook

from helper_functions import unzip_data, create_tensorboard_callback,compare_historys, plot_loss_curves

## Get a text dataset
Kaggle's Introdunction to NLP dataset



In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-09-12 18:24:02--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.128, 74.125.24.128, 172.217.194.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.008s  

2022-09-12 18:24:02 (77.0 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
# Unzip data

unzip_data('nlp_getting_started.zip')

## Visualize a text dataset

In [5]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [6]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
train_df_shuffled = train_df.sample(frac = 1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
len(train_df)

7613

In [11]:
len(test_df)

3263

In [12]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[['text', 'target']][random_index : random_index + 5].itertuples():
  print(row)
  _, text, target = row
  print(f'Target: {target}', '(real disaster)' if target > 0 else '(not real disaster)')
  print(f'Text:\n,{text}\n')
  print(f'----\n')

Pandas(Index=6607, text='DireTube Information \x89ÛÒ Egypt Cyprus and Greece agreed to fightåÊterrorism http://t.co/V6IjxCCD2I http://t.co/YSXhFWMGOD', target=1)
Target: 1 (real disaster)
Text:
,DireTube Information ÛÒ Egypt Cyprus and Greece agreed to fightåÊterrorism http://t.co/V6IjxCCD2I http://t.co/YSXhFWMGOD

----

Pandas(Index=3345, text='Our thoughts are with these local residents! Time for some heavy rain!!! http://t.co/x3g2OX6K8R', target=1)
Target: 1 (real disaster)
Text:
,Our thoughts are with these local residents! Time for some heavy rain!!! http://t.co/x3g2OX6K8R

----

Pandas(Index=6798, text='@itss_selenaluna like a beautiful ass tragedy lol', target=1)
Target: 1 (real disaster)
Text:
,@itss_selenaluna like a beautiful ass tragedy lol

----

Pandas(Index=5870, text='@nbc I wanna see you reboot The Fresh Prince of Bel-Air bring back the original cast and everything &amp; do nothing that will ruin the show.', target=0)
Target: 0 (not real disaster)
Text:
,@nbc I wanna s

### Split data into training and validation sets


In [14]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                              train_df_shuffled['target'].to_numpy(),
                                                              test_size = 0.1,
                                                              random_state = 42)

In [15]:
# Check the lenghts

len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

## Convert text into numbers

Tokenisation - Direct mapping of tokens

Embedding - Create a matrix of feature vectors


### Text vectorisation

In [16]:
train_sentences[:3]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....'],
      dtype=object)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens = None, # limit on the no of tokens (uncommon words get replaced by <OOV>)
                                     standardize = 'lower_and_strip_punctuation', # lowers text and removes punctuation
                                     split = 'whitespace', #split where the white spaces exist
                                     ngrams = None, 
                                     output_mode = 'int',
                                    output_sequence_length = None, # how long the sequences should be
                                    # pad_to_max_tokens = True
                                    )

In [18]:
# Find the average number of tokens in thee trainint twee
ln = (sum([len(i.split()) for i in train_sentences]))/len(train_sentences)
round(ln)

15

In [19]:
# Setup text vectorisation variables

max_vocab_length = 10000
max_length = round(ln)

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [20]:
# Fit the text vectoriser

text_vectorizer.adapt(train_sentences)

In [21]:
sample_sentence = "There's a flood in my street!"

text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
# Choose a random sentence from the training dataset and tokenize it 

random_sentence = random.choice(train_sentences)
print(f'OG sent:\n {random_sentence}\n')

print(f'Vectorized sent:\n {text_vectorizer([random_sentence])}')

OG sent:
 #Insurance: Texas Seeks Comment on Rules for Changes to Windstorm Insurer http://t.co/rb02svlpPu

Vectorized sent:
 [[ 887  758 1479 1219   11  993   10 1314    5  428 1512    1    0    0
     0]]


In [23]:
text_vectorizer([random_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 887,  758, 1479, 1219,   11,  993,   10, 1314,    5,  428, 1512,
           1,    0,    0,    0]])>

In [24]:
# Get the unique words in our vocabulary

words_in_vocab = text_vectorizer.get_vocabulary()

words_in_vocab[:5]


['', '[UNK]', 'the', 'a', 'in']

In [25]:
words_in_vocab[-5:]

['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

In [26]:
len(words_in_vocab)

10000

In [27]:
words_in_vocab[-1]

'paddytomlinson1'

### Create an embedding representation of words

Parameters:

      input_dim = size of vocab
      output_dim = size of the output_sequence_length
      input_length = length of the seq


In [28]:
tf.random.set_seed(42)
from tensorflow.keras import layers

vocab_length = len(words_in_vocab)

embedding = layers.Embedding(input_dim = vocab_length,
                             output_dim = 128,
                             embeddings_initializer = 'uniform',
                             input_length = max_length)

In [29]:
# Get a random sentence from the training set

random_sentence = random.choice(train_sentences)

print(f'Original text:\n {random_sentence}\n')

print('After Embedding:')

# Embed the ranodm sent

sample_embed = embedding(text_vectorizer([random_sentence]))

print(sample_embed)

Original text:
 Reddit: http://t.co/UtuMVaABz6 Lightning sparks new fires in Boise National Forest /r/StormComing

After Embedding:
tf.Tensor(
[[[-0.04901577  0.01449073  0.00180269 ... -0.0090353   0.03036625
   -0.04714844]
  [ 0.03977952 -0.03782602 -0.03646283 ...  0.00236253  0.03332629
    0.02803668]
  [-0.00418826 -0.02975574  0.04156275 ... -0.0264973  -0.01173973
    0.01569256]
  ...
  [ 0.01645621 -0.00589932 -0.01471175 ... -0.02511839  0.00912381
   -0.00024097]
  [ 0.01645621 -0.00589932 -0.01471175 ... -0.02511839  0.00912381
   -0.00024097]
  [ 0.01645621 -0.00589932 -0.01471175 ... -0.02511839  0.00912381
   -0.00024097]]], shape=(1, 15, 128), dtype=float32)


In [30]:
sample_embed

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04901577,  0.01449073,  0.00180269, ..., -0.0090353 ,
          0.03036625, -0.04714844],
        [ 0.03977952, -0.03782602, -0.03646283, ...,  0.00236253,
          0.03332629,  0.02803668],
        [-0.00418826, -0.02975574,  0.04156275, ..., -0.0264973 ,
         -0.01173973,  0.01569256],
        ...,
        [ 0.01645621, -0.00589932, -0.01471175, ..., -0.02511839,
          0.00912381, -0.00024097],
        [ 0.01645621, -0.00589932, -0.01471175, ..., -0.02511839,
          0.00912381, -0.00024097],
        [ 0.01645621, -0.00589932, -0.01471175, ..., -0.02511839,
          0.00912381, -0.00024097]]], dtype=float32)>

In [31]:
# Check out a singe token's embedding

sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.04901577,  0.01449073,  0.00180269,  0.03145752,  0.03001913,
        0.04779543,  0.01454267, -0.04404233,  0.01641065, -0.01868325,
        0.03098175,  0.04230898, -0.04427271, -0.02168292, -0.03067183,
        0.00052283,  0.04506895,  0.03119186,  0.04603002,  0.04919041,
        0.03823203, -0.04824071,  0.04257606, -0.0275    ,  0.04433152,
        0.02838036, -0.04082304, -0.03536421,  0.00921013,  0.00343283,
        0.00741158,  0.02859265, -0.03695261,  0.03345582, -0.02572121,
        0.03614236, -0.02552737, -0.02286865, -0.04795185, -0.04963329,
       -0.04766253,  0.02352193,  0.00826324,  0.04830314, -0.02283541,
       -0.02283167,  0.04145231,  0.02272217,  0.00565696,  0.02288375,
        0.00543882,  0.03858927,  0.03044422, -0.00135201,  0.02223692,
        0.02482006, -0.00802596, -0.0392676 , -0.00523003, -0.04668751,
        0.02673248, -0.02740769, -0.03583468, -0.00852951,  0.0188503 ,
       -0.038638

In [32]:
random_sentence

'Reddit: http://t.co/UtuMVaABz6 Lightning sparks new fires in Boise National Forest /r/StormComing'

## Modelling a text dataet and running a series of experiments

    Model 0 : Naive Bayes (baseline)

    Model 1 : Feed Forward Neural Network (Dense)

    Model 2 : LSTM model (RNN)

    Model 3 : GRU model (RNN)

    Model 4 : Bidirectional LSTM model (RNN)

    Model 5 : 1D CNN

    Model 6:  Tensorflow Hub pretrained Feature Extractor (transfer learning)

    Model 7: Same as model 6 with 10% of training data

### Model 0 : Getting a baseline 


As with all machine learning experiments its important to create a baseline model for benchmarking

> Its common practice to use non_DL as baseline and use DL to see if we can improve upon it

In [33]:
from tensorflow.python.ops.gen_random_ops import Multinomial
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()), # turn our text to numbers
    ('clf', MultinomialNB())     # model the text
])

# Fit the pipe line to the training ata

model_0.fit(train_sentences, train_labels)


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [34]:
# Evaluate our baseline our model

baseline_score = model_0.score(val_sentences, val_labels)

In [35]:
print(f'Our baseline model achieves an accurayc of:{baseline_score*100}%')

Our baseline model achieves an accurayc of:79.26509186351706%


In [36]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [37]:
# Make predictions

baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

Compare our model using the following metrics:

>Accuracy

>Precision

>Recall

>F1-Score

In [38]:
# Function to eval 

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):

  # Accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Precison, Recall, F1-Score
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = 'weighted')
  model_results = {'accuracy': model_accuracy,
                   'precision' : model_precision,
                   'recall': model_recall,
                   'f1': model_f1}
  return model_results

In [39]:
baseline_results = calculate_results(y_true = val_labels,
                                     y_pred = baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1 : A simple dense model

In [40]:
# Create a tensorboard callback
from helper_functions import create_tensorboard_callback

# Createa a directory to save TensorBoard lofs
SAVE_DIR = 'model_logs'

In [41]:
# Build model with Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype = tf.string) # inputs are 1 dimensinal strting
x = text_vectorizer(inputs) # nums to vectors
x = embedding(x) 
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')


In [42]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [43]:
# Compile the model

model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [44]:
# Fit the model

model_history = model_1.fit(x = train_sentences,
                          y = train_labels,
                          epochs = 5,
                          validation_data = (val_sentences, val_labels),
                          callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                   experiment_name = 'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20220912-182407
Epoch 1/5
215/215 [==============================] - 5s 5ms/step - loss: 0.6094 - accuracy: 0.6916 - val_loss: 0.5357 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.4410 - accuracy: 0.8189 - val_loss: 0.4691 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3463 - accuracy: 0.8605 - val_loss: 0.4590 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2848 - accuracy: 0.8923 - val_loss: 0.4641 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2380 - accuracy: 0.9118 - val_loss: 0.4767 - val_accuracy: 0.7874


In [45]:
# Check the results

model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4767 - accuracy: 0.7874


[0.47668465971946716, 0.787401556968689]

In [46]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[0]

array([0.40488204], dtype=float32)

In [47]:
model_1_pred_probs.shape

(762, 1)

In [48]:
# Convert model prediction probs to label formats

model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [49]:
# Calcualte model 1 results
model_1_results = calculate_results(y_true = val_labels,
                            y_pred = model_1_preds)
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7914920592553047,
 'recall': 0.7874015748031497,
 'f1': 0.7846966492209201}

## Visualizing learned embeddings



In [50]:
# Get the vocab from the text vectorization layer

words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab)

10000

In [51]:
words_in_vocab[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

In [52]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [54]:
# Get the wieght matrix of embedding layer

# These are the numerical representation of each token in our training data which have been 

embed_weights = model_1.get_layer('embedding').get_weights()[0]
embed_weights

array([[ 0.00073165,  0.015048  , -0.03425455, ..., -0.04403539,
        -0.0104228 ,  0.01876438],
       [ 0.04135862, -0.03945085, -0.03811941, ...,  0.00464736,
         0.03163552,  0.029283  ],
       [ 0.00684031,  0.05363133, -0.00241554, ..., -0.07082179,
        -0.04750704,  0.01448255],
       ...,
       [-0.03301444, -0.0052493 , -0.04209725, ...,  0.02028764,
         0.00308807,  0.02215792],
       [ 0.00692342,  0.05942352, -0.01975194, ..., -0.06199061,
        -0.01018393,  0.03510419],
       [-0.0372346 ,  0.06267187, -0.07451146, ..., -0.02367218,
        -0.0864333 ,  0.01742155]], dtype=float32)

In [55]:
# Create embedding files
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [56]:
# Download files from colab and upload to projector tool

try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent neural network (RNN's)

> RNN's are useful for sequence data.

> The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of the later input.

> 

In [57]:
2784 *3*4

33408

### Model 2 : LSTM

LSTM = Long Short Term 

Structur of an RNN

    Input(text) -> Tokenize -> Embedding -> Layers(RNNs/Dense) -> Output (label probability)

In [58]:
# Create an LSTM Modle

from tensorflow.keras import layers

inputs = layers.Input(shape=(1, ), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
x = layers.LSTM(64, return_sequences = True)(x) # when you're stacking RNN cells together you need to return sequences
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
x = layers.Dense(64, activation = 'relu')(x)
# print(x.shape)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name = 'model_2_LSTM')

In [59]:
# Get a summary

model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                      

In [60]:
# Compile the model
model_2.compile(loss= 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics =['accuracy'])

In [61]:
# Fit the model

model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       'model_2')])

Saving TensorBoard log files to: model_logs/model_2/20220912-182442
Epoch 1/5
215/215 [==============================] - 10s 20ms/step - loss: 0.2189 - accuracy: 0.9250 - val_loss: 0.5444 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1634 - accuracy: 0.9412 - val_loss: 0.6071 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1282 - accuracy: 0.9520 - val_loss: 0.7574 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1076 - accuracy: 0.9584 - val_loss: 0.9243 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0815 - accuracy: 0.9635 - val_loss: 0.9809 - val_accuracy: 0.7808


In [62]:
model_2_pred_prob = model_2.predict(val_sentences)

In [63]:
model_2_pred_prob[:10]

array([[2.3730553e-03],
       [5.7842934e-01],
       [9.9988604e-01],
       [5.8599312e-02],
       [2.6671814e-05],
       [9.9545348e-01],
       [9.3895912e-01],
       [9.9989414e-01],
       [9.9978155e-01],
       [3.9918759e-01]], dtype=float32)

In [64]:
# Convert model 2 pred probs to labels

model_2_preds = tf.squeeze(tf.round(model_2_pred_prob))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [65]:
# Calcualte model 2 results
model_2_results = calculate_results(y_true = val_labels,
                                    y_pred = model_2_preds)

In [66]:
model_2_results

{'accuracy': 78.08398950131233,
 'precision': 0.7856875884556138,
 'recall': 0.7808398950131233,
 'f1': 0.777681907957685}

In [67]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [68]:
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7914920592553047,
 'recall': 0.7874015748031497,
 'f1': 0.7846966492209201}

### Model_3: GRU

GRU - Gated Recurrent Unit
The GRU cell has similar features to an LSTM cell but has less parameters.

In [69]:
from tensorflow.keras import layers
inputs = layers.Input(shape =(1, ), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# x = layers.LSTM(64, return_sequences = True)(x)
# x = layers.GRU(64)(x)
# x = layers.Dense(64, activation = 'relu')(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name = 'model_3_GRU')

In [70]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [71]:
# Xomile the modle

model_3.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics= ['accuracy'])

# Fit the model

model_3.history = model_3.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       'model_3_Gru')])

Saving TensorBoard log files to: model_logs/model_3_Gru/20220912-182503
Epoch 1/5
215/215 [==============================] - 4s 8ms/step - loss: 0.1620 - accuracy: 0.9377 - val_loss: 0.6392 - val_accuracy: 0.7795
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0887 - accuracy: 0.9672 - val_loss: 0.7860 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0717 - accuracy: 0.9721 - val_loss: 1.0992 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0630 - accuracy: 0.9739 - val_loss: 1.0155 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0519 - accuracy: 0.9758 - val_loss: 1.2545 - val_accuracy: 0.7782


In [72]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[4.7421441e-04],
       [7.2494882e-01],
       [9.9988055e-01],
       [9.6555941e-02],
       [6.1819919e-05],
       [9.9976522e-01],
       [9.6143895e-01],
       [9.9993181e-01],
       [9.9987459e-01],
       [9.8283893e-01]], dtype=float32)

In [73]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))

In [74]:
model_3_results = calculate_results(y_true = val_labels,
                                    y_pred = model_3_preds)

In [75]:
model_3_results

{'accuracy': 77.82152230971128,
 'precision': 0.7791991586469295,
 'recall': 0.7782152230971129,
 'f1': 0.7766208088282468}

In [76]:
model_2_results

{'accuracy': 78.08398950131233,
 'precision': 0.7856875884556138,
 'recall': 0.7808398950131233,
 'f1': 0.777681907957685}

In [77]:
model_2_results

{'accuracy': 78.08398950131233,
 'precision': 0.7856875884556138,
 'recall': 0.7808398950131233,
 'f1': 0.777681907957685}

In [78]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 4: Bidirectional RNN

Normal RNN's go from left to right to left.

Bidrectional RNN goes from right to left aswell as left to right

In [79]:
# Bulid a bidirectional RNN in tensorflow

from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_4 =  tf.keras.Model(inputs, outputs, name = 'model_4_bidirectional')

In [80]:
# Model 4 summary

model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [81]:
# Compile the model

model_4.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])


In [82]:
# Fit the model

model_4_history = model_4.fit(train_sentences, train_labels, epochs  =5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       'Model_4_Bidirectioanl')])

Saving TensorBoard log files to: model_logs/Model_4_Bidirectioanl/20220912-182516
Epoch 1/5
215/215 [==============================] - 5s 12ms/step - loss: 0.1096 - accuracy: 0.9692 - val_loss: 0.9659 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0519 - accuracy: 0.9762 - val_loss: 1.2894 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0458 - accuracy: 0.9791 - val_loss: 1.3640 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0431 - accuracy: 0.9806 - val_loss: 1.0325 - val_accuracy: 0.7533
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0454 - accuracy: 0.9796 - val_loss: 1.4523 - val_accuracy: 0.7625


In [83]:
# Make prediction 

model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[1.1540219e-04],
       [6.5747565e-01],
       [9.9995530e-01],
       [1.9027892e-01],
       [5.4347520e-06],
       [9.9795640e-01],
       [5.0462466e-01],
       [9.9997413e-01],
       [9.9994719e-01],
       [8.9158070e-01]], dtype=float32)

In [84]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [85]:
# Calculate the results of our bidirectional model

model_4_results = calculate_results(y_true = val_labels,
                                    y_pred = model_4_preds)

In [86]:
print( model_4_results, '\n', model_3_results, '\n', model_2_results, '\n', model_1_results, '\n', baseline_results)

{'accuracy': 76.24671916010499, 'precision': 0.7677266148183088, 'recall': 0.7624671916010499, 'f1': 0.7585330817821493} 
 {'accuracy': 77.82152230971128, 'precision': 0.7791991586469295, 'recall': 0.7782152230971129, 'f1': 0.7766208088282468} 
 {'accuracy': 78.08398950131233, 'precision': 0.7856875884556138, 'recall': 0.7808398950131233, 'f1': 0.777681907957685} 
 {'accuracy': 78.74015748031496, 'precision': 0.7914920592553047, 'recall': 0.7874015748031497, 'f1': 0.7846966492209201} 
 {'accuracy': 79.26509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


## Convolutional Neural Network for Text (and other type of sequences)

>Images are 2D (Conv2D)

>Text are 1D (Conv1D)

Typical Structure of Conv2D:

    Inputs(test) --> Tokenisatin --> Embedding --> Conv1D --> Pooling --> Outputs

### Model_5: Conv1D

####Brief Overview

In [90]:
# Test out our embedding layer, Conv1D layer and max pooling

embedding_test = embedding(text_vectorizer(['this is a test sentence'])) # Turn into Embedding
conv_1d = layers.Conv1D(filters = 64,
                        kernel_size = 5,
                        activation = 'relu',
                        padding = 'same')

In [94]:
conv_1d_output = conv_1d(embedding_text)
conv_1d_output[0]

<tf.Tensor: shape=(15, 64), dtype=float32, numpy=
array([[0.        , 0.00556169, 0.        , 0.        , 0.        ,
        0.03012613, 0.        , 0.        , 0.        , 0.        ,
        0.03184682, 0.00337346, 0.03176581, 0.        , 0.        ,
        0.05261538, 0.0076737 , 0.        , 0.        , 0.        ,
        0.        , 0.03427724, 0.06237193, 0.        , 0.01206259,
        0.00637787, 0.01392604, 0.        , 0.00983667, 0.01492696,
        0.        , 0.        , 0.03280503, 0.01289649, 0.        ,
        0.00830543, 0.01175603, 0.        , 0.        , 0.        ,
        0.0314199 , 0.04730297, 0.01655575, 0.05854502, 0.02888556,
        0.        , 0.0541241 , 0.        , 0.02514843, 0.        ,
        0.        , 0.08583094, 0.        , 0.        , 0.        ,
        0.00904964, 0.        , 0.        , 0.        , 0.00457649,
        0.00718857, 0.00888377, 0.02674362, 0.04487272],
       [0.00660127, 0.03351861, 0.        , 0.        , 0.        ,
        0

In [95]:
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # Get the most important feature
max_pool_output[0]

<tf.Tensor: shape=(64,), dtype=float32, numpy=
array([0.01055856, 0.06478687, 0.01472261, 0.00102557, 0.01566675,
       0.03012613, 0.07528871, 0.08397712, 0.05726048, 0.04120113,
       0.03184682, 0.03629782, 0.03176581, 0.01802219, 0.0090463 ,
       0.08946073, 0.0539113 , 0.0007759 , 0.02082286, 0.04391163,
       0.01326357, 0.1017101 , 0.06237193, 0.00535074, 0.01206259,
       0.02422307, 0.01392604, 0.0301909 , 0.07318126, 0.08802286,
       0.03549784, 0.09100123, 0.03280503, 0.0311834 , 0.        ,
       0.04703682, 0.06709199, 0.0070895 , 0.04661107, 0.03307819,
       0.0314199 , 0.07150989, 0.04907646, 0.06292878, 0.02888556,
       0.03862662, 0.0541241 , 0.02986691, 0.02514843, 0.0464445 ,
       0.04625812, 0.08583094, 0.03607662, 0.06253541, 0.02931677,
       0.09385747, 0.03912998, 0.0143307 , 0.05695824, 0.05069713,
       0.07943203, 0.08708324, 0.06441031, 0.0821481 ], dtype=float32)>

In [97]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.0161108 , -0.08999023, -0.02478556, ..., -0.01295709,
         -0.06609146,  0.00225566],
        [-0.03420832,  0.01560463, -0.00260672, ..., -0.03109274,
         -0.01397596,  0.08134957],
        [-0.04404969, -0.0465281 , -0.02806975, ..., -0.01460602,
          0.01121991,  0.05237249],
        ...,
        [-0.00913715, -0.01031336, -0.01518035, ..., -0.01903374,
         -0.01438572,  0.02821921],
        [-0.00913715, -0.01031336, -0.01518035, ..., -0.01903374,
         -0.01438572,  0.02821921],
        [-0.00913715, -0.01031336, -0.01518035, ..., -0.01903374,
         -0.01438572,  0.02821921]]], dtype=float32)>

In [98]:
conv_1d_output

<tf.Tensor: shape=(1, 15, 64), dtype=float32, numpy=
array([[[0.        , 0.00556169, 0.        , 0.        , 0.        ,
         0.03012613, 0.        , 0.        , 0.        , 0.        ,
         0.03184682, 0.00337346, 0.03176581, 0.        , 0.        ,
         0.05261538, 0.0076737 , 0.        , 0.        , 0.        ,
         0.        , 0.03427724, 0.06237193, 0.        , 0.01206259,
         0.00637787, 0.01392604, 0.        , 0.00983667, 0.01492696,
         0.        , 0.        , 0.03280503, 0.01289649, 0.        ,
         0.00830543, 0.01175603, 0.        , 0.        , 0.        ,
         0.0314199 , 0.04730297, 0.01655575, 0.05854502, 0.02888556,
         0.        , 0.0541241 , 0.        , 0.02514843, 0.        ,
         0.        , 0.08583094, 0.        , 0.        , 0.        ,
         0.00904964, 0.        , 0.        , 0.        , 0.00457649,
         0.00718857, 0.00888377, 0.02674362, 0.04487272],
        [0.00660127, 0.03351861, 0.        , 0.        , 0.  

In [99]:
max_pool_output

<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
array([[0.01055856, 0.06478687, 0.01472261, 0.00102557, 0.01566675,
        0.03012613, 0.07528871, 0.08397712, 0.05726048, 0.04120113,
        0.03184682, 0.03629782, 0.03176581, 0.01802219, 0.0090463 ,
        0.08946073, 0.0539113 , 0.0007759 , 0.02082286, 0.04391163,
        0.01326357, 0.1017101 , 0.06237193, 0.00535074, 0.01206259,
        0.02422307, 0.01392604, 0.0301909 , 0.07318126, 0.08802286,
        0.03549784, 0.09100123, 0.03280503, 0.0311834 , 0.        ,
        0.04703682, 0.06709199, 0.0070895 , 0.04661107, 0.03307819,
        0.0314199 , 0.07150989, 0.04907646, 0.06292878, 0.02888556,
        0.03862662, 0.0541241 , 0.02986691, 0.02514843, 0.0464445 ,
        0.04625812, 0.08583094, 0.03607662, 0.06253541, 0.02931677,
        0.09385747, 0.03912998, 0.0143307 , 0.05695824, 0.05069713,
        0.07943203, 0.08708324, 0.06441031, 0.0821481 ]], dtype=float32)>

###Building the model

In [100]:
# Create 1-dimenstioanl convolutional layer to model sequence

from tensorflow.keras import layers

inputs = layers.Input(shape = (1, ), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64,
                  kernel_size = 5,
                  activation = 'relu',
                  padding = 'valid',
                  strides = 1)(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(64, activation = 'relu')(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs, name = 'model_5_Conv1D')

# Compile the model
model_5.compile(loss ='binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [101]:
# Model 5 summary
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_3 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)              

In [102]:
# Fit the model

model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs = 5 ,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       'Conv1D')])

Saving TensorBoard log files to: model_logs/Conv1D/20220912-182724
Epoch 1/5
215/215 [==============================] - 3s 6ms/step - loss: 0.1200 - accuracy: 0.9604 - val_loss: 0.8632 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0732 - accuracy: 0.9726 - val_loss: 1.0250 - val_accuracy: 0.7638
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0610 - accuracy: 0.9764 - val_loss: 1.1022 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0544 - accuracy: 0.9780 - val_loss: 1.1741 - val_accuracy: 0.7651
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0501 - accuracy: 0.9787 - val_loss: 1.2165 - val_accuracy: 0.7585


In [103]:
# Make some predictions with out Conv1D model

model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

array([[8.8989057e-02],
       [7.0117712e-01],
       [9.9994779e-01],
       [8.4438086e-02],
       [1.8098977e-07],
       [9.9584019e-01],
       [9.7653610e-01],
       [9.9997425e-01],
       [9.9999917e-01],
       [8.7088335e-01]], dtype=float32)

In [104]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))

In [105]:
model_5_results = calculate_results(y_true = val_labels,
                            y_pred = model_5_preds)

In [106]:
print(model_5_results, '\n', model_4_results, '\n', model_3_results, '\n', model_2_results, '\n', model_1_results, '\n', baseline_results)

{'accuracy': 75.8530183727034, 'precision': 0.7593999956661763, 'recall': 0.7585301837270341, 'f1': 0.7566051475454213} 
 {'accuracy': 76.24671916010499, 'precision': 0.7677266148183088, 'recall': 0.7624671916010499, 'f1': 0.7585330817821493} 
 {'accuracy': 77.82152230971128, 'precision': 0.7791991586469295, 'recall': 0.7782152230971129, 'f1': 0.7766208088282468} 
 {'accuracy': 78.08398950131233, 'precision': 0.7856875884556138, 'recall': 0.7808398950131233, 'f1': 0.777681907957685} 
 {'accuracy': 78.74015748031496, 'precision': 0.7914920592553047, 'recall': 0.7874015748031497, 'f1': 0.7846966492209201} 
 {'accuracy': 79.26509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


## Model_6 : TensorFlow Hub Pretrained Sentence Encoder

In [107]:
import tensorflow_hub as hub
embed = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
embed_samples = embed([sample_sentence, 'When you can the universal sentence encoder on a sentence, it turns it into numbers'])
print(embed_samples[0][:50])

tf.Tensor(
[-0.01157028  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680984  0.05589836 -0.0106873  -0.00597291  0.00639323 -0.01819518
  0.00030813  0.09105888  0.05874644 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865347 -0.04209306  0.02678981  0.03011006  0.00321069
 -0.00337973 -0.04787357  0.0226672  -0.00985925 -0.04063613 -0.01292092
 -0.04666384  0.05630299 -0.03949255  0.00517686  0.02495829 -0.0701444
  0.02871508  0.04947684 -0.00633979 -0.08960192  0.02807118 -0.00808364
 -0.01360602  0.0599865  -0.10361787 -0.05195374  0.00232954 -0.02332531
 -0.03758105  0.03327728], shape=(50,), dtype=float32)


In [108]:
embed_samples

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[-0.01157028,  0.0248591 ,  0.02878048, ..., -0.00186124,
         0.02315826, -0.01485021],
       [ 0.03485871, -0.08845595, -0.01677877, ..., -0.02750707,
         0.03230237, -0.00820089]], dtype=float32)>

In [109]:
# Create a keras layer using the USE Pretrained layer form tensorflow hub

sentence_encoder_layer = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                                       input_shape = [],
                                       dtype = tf.string,
                                       name = 'USE')


In [110]:
# Create model using the Sequential API:

model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation = 'relu'),
    layers.Dense(1, activation = 'sigmoid')
], name = 'model_6_USE')

In [111]:
# Compile the model

model_6.compile(loss = 'binary_crossentropy',
            optimizer = tf.keras.optimizers.Adam(),
            metrics = ['accuracy'])

In [112]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [113]:
# Train a classifer on top of USE Pretrained embeddings

model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       'tf_hub_sent_encoder')])

Saving TensorBoard log files to: model_logs/tf_hub_sent_encoder/20220912-182758
Epoch 1/5
215/215 [==============================] - 5s 14ms/step - loss: 0.5033 - accuracy: 0.7838 - val_loss: 0.4466 - val_accuracy: 0.7940
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4142 - accuracy: 0.8142 - val_loss: 0.4350 - val_accuracy: 0.8110
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3994 - accuracy: 0.8222 - val_loss: 0.4308 - val_accuracy: 0.8123
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3920 - accuracy: 0.8269 - val_loss: 0.4265 - val_accuracy: 0.8176
Epoch 5/5
215/215 [==============================] - 2s 12ms/step - loss: 0.3858 - accuracy: 0.8291 - val_loss: 0.4281 - val_accuracy: 0.8136


In [114]:
# Make predictions with USE tf hub models

model_6_pred_probs = model_6.predict(val_sentences)

In [115]:
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))

In [116]:
# Calculate model 6 performance

model_6_results = calculate_results(y_true = val_labels,
                                    y_pred = model_6_preds)

In [117]:
print(model_6_results, '\n', model_5_results, '\n', model_4_results, '\n', model_3_results, '\n', model_2_results, '\n', model_1_results, '\n', baseline_results)

{'accuracy': 81.36482939632546, 'precision': 0.8160661319598898, 'recall': 0.8136482939632546, 'f1': 0.8120618868299143} 
 {'accuracy': 75.8530183727034, 'precision': 0.7593999956661763, 'recall': 0.7585301837270341, 'f1': 0.7566051475454213} 
 {'accuracy': 76.24671916010499, 'precision': 0.7677266148183088, 'recall': 0.7624671916010499, 'f1': 0.7585330817821493} 
 {'accuracy': 77.82152230971128, 'precision': 0.7791991586469295, 'recall': 0.7782152230971129, 'f1': 0.7766208088282468} 
 {'accuracy': 78.08398950131233, 'precision': 0.7856875884556138, 'recall': 0.7808398950131233, 'f1': 0.777681907957685} 
 {'accuracy': 78.74015748031496, 'precision': 0.7914920592553047, 'recall': 0.7874015748031497, 'f1': 0.7846966492209201} 
 {'accuracy': 79.26509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


## Model_7 : TF Hub pretrained but with 10% of training data

Transfer Learning really helps when you dont have a large dataset

In [118]:
# Create subsets of 10% of the training data

train_10_percent = train_df_shuffled[['text', 'target']].sample(frac=0.1, random_state =42)
train_10_percent.head()

,text,target
4955,DFR EP016 Monthly Meltdown - On Dnbheaven 2015...,0
584,FedEx no longer to transport bioterror germs i...,0
7411,Gunmen kill four in El Salvador bus attack: Su...,1
5950,@camilacabello97 Internally and externally scr...,1
5541,Radiation emergency #preparedness starts with ...,1


In [119]:
train_sents_10_percent = train_10_percent['text'].to_list()
train_labels_10_percent = train_10_percent['target'].to_list()

In [120]:
train_10_percent['target'].value_counts()

0    413
1    348
Name: target, dtype: int64

In [121]:
train_df_shuffled['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

To recreate a model same as the previous model you created you can use the tf.keras clone model

In [125]:
# Lets build a model the same as model_6

model_7 = tf.keras.models.clone_model(model_6)

# Compile the model

model_7.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [123]:
model_7.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [130]:
# Fit the model
model_7.fit(train_sents_10_percent, train_labels_10_percent,
            validation_data = (val_sentences, val_labels),
            epochs = 5,
            callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                     'tf_hub_model_7_10%')])

Saving TensorBoard log files to: model_logs/model_7_10%/20220912-185412
Epoch 1/5
24/24 [==============================] - 4s 38ms/step - loss: 0.6683 - accuracy: 0.6965 - val_loss: 0.6282 - val_accuracy: 0.7940
Epoch 2/5
24/24 [==============================] - 1s 25ms/step - loss: 0.5927 - accuracy: 0.7950 - val_loss: 0.5474 - val_accuracy: 0.7979
Epoch 3/5
24/24 [==============================] - 1s 22ms/step - loss: 0.5157 - accuracy: 0.7950 - val_loss: 0.4793 - val_accuracy: 0.8045
Epoch 4/5
24/24 [==============================] - 1s 22ms/step - loss: 0.4601 - accuracy: 0.8081 - val_loss: 0.4353 - val_accuracy: 0.8150
Epoch 5/5
24/24 [==============================] - 1s 26ms/step - loss: 0.4243 - accuracy: 0.8160 - val_loss: 0.4055 - val_accuracy: 0.8241


In [132]:
# Make predictions with the models trained on 10% of the data

model_7_pred_probs = model_7.predict(val_sentences)
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))

In [134]:
# Eval moddel 7

model_7_results = calculate_results(y_true = val_labels,
                                    y_pred = model_7_preds)